In [ ]:
import datajoint as dj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [ ]:
dj.config['database.host'] = 'aeon-db'
dj.conn()

In [ ]:
dj.config['display.limit'] = 10
dj.config['display.width'] = 30

In [ ]:
subject = dj.create_virtual_module('subject', 'aeon_test_subject')
lab = dj.create_virtual_module('lab', 'aeon_test_lab')
experiment = dj.create_virtual_module('experiment', 'aeon_test_experiment')
tracking = dj.create_virtual_module('tracking', 'aeon_test_tracking')

In [ ]:
# Retrieve and plot subject's trajectory in response to pellet trigger events

# -- subject of interest
subject_key = {'subject': 'BAA-1099791'}

# -- all epochs this subject is in the arena
subject_epochs = experiment.Epoch.Subject & subject_key

# epoch starts/ends
epoch_starts, epoch_ends = subject_epochs.fetch('epoch_start', 'epoch_end')

In [ ]:
# subject's position data in the epochs
timestamps, position_x, position_y, area = (tracking.SubjectPosition & subject_epochs).fetch(
    'timestamps', 'position_x', 'position_y', 'area', order_by='epoch_start')

# stack and structure in pandas DataFrame
position = pd.DataFrame(dict(x=np.hstack(position_x),
                             y=np.hstack(position_y),
                             area=np.hstack(area)),
                        index=np.hstack(timestamps))

# remove position data where area >= 2000
position = position[position.area < 2000]

In [ ]:
# -- all TriggerPellet events from patch
patch_key = (experiment.ExperimentFoodPatch - experiment.ExperimentFoodPatch.RemovalTime
             & 'food_patch_description = "Patch1"').fetch1('KEY')

patch_events = (experiment.FoodPatchEvent * experiment.EventType
                * experiment.ExperimentFoodPatch
                & patch_key & 'event_type = "TriggerPellet"')

# -- all TriggerPellet events from patch during times this animal is in the arena
time_restriction = [f'event_time BETWEEN "{s}" AND "{e}"'
                    for s, e in zip(epoch_starts, epoch_ends)]
patch_subject_events = patch_events & time_restriction
patch_subject_events_times = patch_subject_events.fetch('event_time', order_by='event_time')

In [ ]:
patches_x, patches_y = (experiment.ExperimentFoodPatch * experiment.ExperimentFoodPatch.Position
                    - experiment.ExperimentFoodPatch.RemovalTime).fetch(
    'food_patch_position_x', 'food_patch_position_y')


In [ ]:
# plot subject's path and pellet-event aligned path

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
ax.invert_yaxis()
ax.plot(position.x, position.y, 'k', alpha=0.2)
for x, y in zip(patches_x, patches_y):
    ax.plot(x, y, '.r', markersize=50)

for trial in patch_subject_events_times:                   # for each pellet delivery
    before = trial - datetime.timedelta(seconds=15)         # get the previous 15 seconds
    path = position[before:trial]                           # get position data in the time before pellet delivery
    ax.plot(path.x, path.y)                                 # plot path traces preceding pellet delivery


In [ ]:
# epochs contain the events
event_time_restriction = [f'"{event}" BETWEEN epoch_start AND epoch_end'
                    for event in patch_subject_events_times]
event_subject_epochs = subject_epochs & event_time_restriction

    